In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [52]:
df_lung = pd.read_csv('/home/davidhenslovitz/Galvanize/ZNAHealth/CSV/RESPIR.csv', converters={'State-county recode': lambda x: str(x), 'CS Schema v0204+': lambda x: str(x)})

/home/davidhenslovitz/anaconda2/envs/py3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7,24,25,54,86) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [53]:
df_lung = df_lung.replace(r'\s+', np.nan, regex=True)

In [54]:
df_lung = df_lung[df_lung['CS Schema v0204+']=='063']

In [55]:
df_lung.shape

(608048, 132)

In [56]:
# df_lung["state"] = df_lung["State-county recode"].str[0:2]
# df_lung["county"] = df_lung["State-county recode"].str[2:5]

In [57]:
df_lung = df_lung[['Race/Ethnicity','Sex','Age at diagnosis', 'Year of diagnosis', 'Year of Birth','CS Schema v0204+','Race recode (White, Black, Other)','Race recode (W, B, AI, API)','Origin recode NHIA (Hispanic, Non-Hisp)','State-county recode']]

In [58]:
df_lung.head()

,Race/Ethnicity,Sex,Age at diagnosis,Year of diagnosis,Year of Birth,CS Schema v0204+,"Race recode (White, Black, Other)","Race recode (W, B, AI, API)","Origin recode NHIA (Hispanic, Non-Hisp)",State-county recode
0,1,1,81,1995,1914,063,1,1,0,09009
1,1,1,88,1973,1884,063,1,1,0,09009
2,1,1,76,1978,1902,063,1,1,0,09009
3,1,2,64,1975,1910,063,1,1,0,09001
4,1,2,66,1991,1924,063,1,1,0,09009


In [59]:
df_lung.loc[ df_lung['Age at diagnosis'].apply(pd.to_numeric) <= 30, 'Age'] = '0'
df_lung.loc[(df_lung['Age at diagnosis'].apply(pd.to_numeric) > 30) & (df_lung['Age at diagnosis'].apply(pd.to_numeric) <= 60), 'Age'] = '1'
df_lung.loc[ df_lung['Age at diagnosis'].apply(pd.to_numeric) > 60, 'Age'] = '2'
df_lung.shape

(608048, 11)

In [100]:
#df_lung['Age at diagnosis'] = df_lung['Age at diagnosis'].astype(int)
df_grouped = df_lung.groupby(['State-county recode', 'Race recode (White, Black, Other)','Sex','Age','Year of diagnosis']).size().reset_index(name='Count')
df_grouped.head()
#df_grouped.head(50)
#pd.DataFrame(df_grouped).reset_index().head()
# df_grouped.pivot(index=df_grouped.index.get_level_values('State-county recode'), columns='Year of diagnosis', values='Count')

,State-county recode,"Race recode (White, Black, Other)",Sex,Age,Year of diagnosis,Count
0,06001,1,1,0,1974,1
1,06001,1,1,0,1979,1
2,06001,1,1,0,1983,1
3,06001,1,1,0,1988,1
4,06001,1,1,0,1989,1


In [101]:
df_grouped['Combined'] = df_grouped.Age.astype(str)+df_grouped['State-county recode'].astype(str)+df_grouped['Race recode (White, Black, Other)'].astype(str)+df_grouped.Sex.astype(str)
df_grouped = df_grouped.pivot(index='Combined',columns='Year of diagnosis',values='Count').reset_index()


In [105]:
df_grouped.shape

(1949, 43)

In [117]:
df_grouped.drop([1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999], axis=1,inplace=True)

In [118]:
df_grouped.head()

Year of diagnosis,Combined,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,Age,State-county recode,"Race recode (White, Black, Other)",Sex
0,00600111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,NaN,3.0,1.0,NaN,0,06001,1,1
1,00600112,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0,06001,1,2
2,00600121,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,06001,2,1
3,00600122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0,06001,2,2
4,00600131,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0,06001,3,1


In [160]:
df_grouped['Age']=df_grouped.Combined.str[:1]
df_grouped['State-county recode']=df_grouped.Combined.str[1:6]
df_grouped['Race recode (White, Black, Other)']=df_grouped.Combined.str[6:7]
df_grouped['Sex']=df_grouped.Combined.str[7:]
#df_grouped.groupby('State-county recode')[2010,2011,2012,2013,2014].sum()
df_grouped[df_grouped.Age=='1'].head()

Year of diagnosis,Combined,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,Age,State-county recode,"Race recode (White, Black, Other)",Sex
228,10600111,55.0,48.0,48.0,53.0,39.0,46.0,45.0,37.0,32.0,34.0,27.0,26.0,35.0,24.0,18.0,1,06001,1,1
229,10600112,34.0,44.0,46.0,52.0,30.0,43.0,44.0,37.0,36.0,45.0,21.0,28.0,21.0,25.0,33.0,1,06001,1,2
230,10600121,38.0,23.0,29.0,22.0,16.0,25.0,20.0,21.0,18.0,25.0,14.0,23.0,22.0,13.0,11.0,1,06001,2,1
231,10600122,26.0,23.0,27.0,23.0,22.0,19.0,26.0,20.0,18.0,28.0,22.0,21.0,26.0,23.0,15.0,1,06001,2,2
232,10600131,8.0,12.0,14.0,22.0,19.0,19.0,9.0,14.0,15.0,21.0,23.0,15.0,20.0,21.0,22.0,1,06001,3,1


In [146]:
#df_grouped[[2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014]].sum().sum()

In [147]:
df_grouped.shape

(1949, 20)

In [49]:
df_grouped = pd.DataFrame(df_grouped).reset_index()

In [50]:
df_grouped.head()

,State-county recode,Year of diagnosis,Count
0,06001,1973,479
1,06001,1974,544
2,06001,1975,540
3,06001,1976,558
4,06001,1977,626


In [57]:
#df_grouped.pivot(index=df_grouped.index.get_level_values(0), columns=df_grouped.index.get_level_values(1), values='Count')
df_grouped.pivot(index='State-county recode', columns='Year of diagnosis', values='Count').shape


(209, 42)

In [123]:
population = pd.read_table('/home/davidhenslovitz/Galvanize/ZNAHealth/SEER_1973_2014_TEXTDATA/populations/white_black_other/yr1973_2014.seer9/singleages.txt',header=None)

In [124]:
population1 = population[:1422890]
population2 = population[1422890:]

In [125]:
indices = [(0,4),(4,6),(6,8),(8,11),(11,13),(13,14),(14,15),(15,16),(16,18),(18,28)]
dct={}
for num in range(len(population1)):
    string = ""
    x = population1.iloc[num][0]
    for i in range(len(indices)):
        string += x[indices[i][0]:indices[i][1]] + ", "
    dct[num] = string[:-2]

In [126]:
indices = [(0,4),(4,6),(6,8),(8,11),(11,13),(13,14),(14,15),(15,16),(16,18),(18,28)]
dct2={}
for num in range(len(population2)):
    string = ""
    x = population2.iloc[num][0]
    for i in range(len(indices)):
        string += x[indices[i][0]:indices[i][1]] + ", "
    dct2[num] = string[:-2]

In [127]:
clean_df = pd.DataFrame(columns=['Year','State postal abbr','State FIPS','County FIPS','Registry','Race','Origin','Sex','Age','Population'])

In [128]:
for i, k in enumerate(dct.values()):
    dct[i] = k.split(', ')

In [129]:
for i, k in enumerate(dct2.values()):
    dct2[i] = k.split(', ')

In [130]:
df1 = pd.DataFrame.from_dict(dct, orient='index')
df2 = pd.DataFrame.from_dict(dct2, orient='index')
df_new = pd.concat([df1, df2])
df_new.head()

,0,1,2,3,4,5,6,7,8,9
0,1973,CA,06,001,01,1,9,1,00,00005529.0
1,1973,CA,06,001,01,1,9,1,01,00005638.0
2,1973,CA,06,001,01,1,9,1,02,00006070.0
3,1973,CA,06,001,01,1,9,1,03,00005993.0
4,1973,CA,06,001,01,1,9,1,04,00005876.0


In [131]:
df_new.columns = clean_df.columns
df_new['State-county recode']=df_new['State FIPS']+df_new['County FIPS']
df_new.Population = df_new.Population.apply(pd.to_numeric)

In [132]:
df_new.loc[ df_new.Age.apply(pd.to_numeric) <= 30, 'Age'] = 0
df_new.loc[(df_new.Age.apply(pd.to_numeric) > 30) & (df_new.Age.apply(pd.to_numeric) <= 60), 'Age'] = 1
df_new.loc[ df_new.Age.apply(pd.to_numeric) > 60, 'Age'] = 2

In [133]:
pop_binned = pd.DataFrame(df_new.groupby(['Age','State-county recode','Race','Sex','Year'])['Population'].sum().reset_index())

In [134]:
#pop_binned.pivot(pop_binned.index,'Year', 'Population')
pop_binned.head()

,Age,State-county recode,Race,Sex,Year,Population
0,0,06001,1,1,1973,228546.0
1,0,06001,1,1,1974,224455.0
2,0,06001,1,1,1975,219655.0
3,0,06001,1,1,1976,217524.0
4,0,06001,1,1,1977,213549.0


In [155]:
pop_binned['Combined'] = pop_binned.Age.astype(str)+pop_binned['State-county recode'].astype(str)+pop_binned.Race.astype(str)+pop_binned.Sex.astype(str)
pop_binned.Year = pop_binned.Year.astype(int)
pop_final = pop_binned.pivot(index='Combined',columns='Year',values='Population').reset_index()

In [156]:
pop_final['Age']=pop_final.Combined.str[:1]
pop_final['State-county recode']=pop_final.Combined.str[1:6]
pop_final['Race']=pop_final.Combined.str[6:7]
pop_final['Sex']=pop_final.Combined.str[7:]
pop_final.head(20)

Year,Combined,1973,1974,1975,1976,1977,1978,1979,1980,1981,...,2009,2010,2011,2012,2013,2014,Age,State-county recode,Race,Sex
0,00600111,228546.0,224455.0,219655.0,217524.0,213549.0,207233.0,201921.0,196881.0,196914.0,...,174513.0,173527.0,173118.0,173497.0,174683.0,176119.0,0,06001,1,1
1,00600112,217479.0,213759.0,209453.0,207872.0,205423.0,200733.0,194391.0,190274.0,189243.0,...,164847.0,164113.0,164198.0,165541.0,166613.0,168250.0,0,06001,1,2
2,00600121,51954.0,52702.0,53518.0,54472.0,55116.0,55433.0,55519.0,57332.0,57690.0,...,46753.0,46830.0,46334.0,46023.0,45465.0,45188.0,0,06001,2,1
3,00600122,53459.0,54499.0,55589.0,56806.0,57885.0,58651.0,58710.0,60657.0,60743.0,...,47839.0,47872.0,47305.0,46849.0,46059.0,45269.0,0,06001,2,2
4,00600131,19299.0,20441.0,21652.0,22948.0,24149.0,25246.0,26286.0,28250.0,31082.0,...,97540.0,98824.0,99912.0,101771.0,104061.0,106111.0,0,06001,3,1
5,00600132,18599.0,19758.0,20973.0,22256.0,23534.0,24746.0,25670.0,27770.0,30521.0,...,96726.0,97944.0,99220.0,101035.0,103481.0,106239.0,0,06001,3,2
6,00601311,133315.0,132416.0,131568.0,132582.0,133928.0,133146.0,134187.0,134632.0,134130.0,...,150310.0,150334.0,150317.0,150300.0,150742.0,150932.0,0,06013,1,1
7,00601312,130682.0,129561.0,128557.0,129563.0,131031.0,130329.0,131314.0,131690.0,130937.0,...,142398.0,142841.0,143369.0,143669.0,144366.0,144870.0,0,06013,1,2
8,00601321,14545.0,14952.0,15422.0,15864.0,16315.0,16573.0,16983.0,17666.0,17750.0,...,26819.0,27243.0,27207.0,27146.0,27486.0,27658.0,0,06013,2,1
9,00601322,15108.0,15526.0,16006.0,16531.0,17101.0,17467.0,17973.0,18715.0,18749.0,...,27172.0,27510.0,27424.0,27238.0,27370.0,27547.0,0,06013,2,2


In [157]:
pop_final.drop([1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999], axis=1,inplace=True)
#pop_final.drop(['1973','1974'],axis=1)

In [158]:
pop_final.head()

Year,Combined,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,Age,State-county recode,Race,Sex
0,00600111,190556.0,189925.0,184765.0,179971.0,176330.0,173623.0,172178.0,172625.0,173977.0,174513.0,173527.0,173118.0,173497.0,174683.0,176119.0,0,06001,1,1
1,00600112,176131.0,175668.0,171497.0,167961.0,164704.0,162572.0,161498.0,161882.0,163760.0,164847.0,164113.0,164198.0,165541.0,166613.0,168250.0,0,06001,1,2
2,00600121,54165.0,52787.0,50726.0,49184.0,48084.0,47098.0,46640.0,46417.0,46513.0,46753.0,46830.0,46334.0,46023.0,45465.0,45188.0,0,06001,2,1
3,00600122,56741.0,54858.0,52680.0,51126.0,49837.0,48524.0,47940.0,47517.0,47504.0,47839.0,47872.0,47305.0,46849.0,46059.0,45269.0,0,06001,2,2
4,00600131,81300.0,86041.0,87293.0,88202.0,88733.0,89747.0,90858.0,92684.0,95271.0,97540.0,98824.0,99912.0,101771.0,104061.0,106111.0,0,06001,3,1


In [3]:
df_lung = pd.read_csv('/home/davidhenslovitz/Galvanize/ZNAHealth/CSV/RESPIR.csv', converters={'State-county recode': lambda x: str(x), 'CS Schema v0204+': lambda x: str(x)})

NameError: name 'pd' is not defined

In [1]:
df_lung = df_lung.replace(r'\s+', np.nan, regex=True)

NameError: name 'df_lung' is not defined

In [ ]:
df_lung = df_lung[df_lung['CS Schema v0204+']=='063']